Importing libraries

In [1]:
import pygame
import neat
import time
import os
import random

pygame 2.1.0 (SDL 2.0.16, Python 3.9.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


Defining the display window

In [2]:
WIN_WIDTH, WIN_HEIGHT = 500, 800
win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
pygame.display.set_caption("Flappy Bird")

Getting and defining all surfaces (texts/images) as constants

In [3]:
BIRD_IMGS = [pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird1.png"))), 
                pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird2.png"))),
                pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird3.png")))]

PIPE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "pipe.png")))
BASE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "base.png")))
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bg.png")))

#frame rate for the game i.e. the #times the display will be refreshed in one second
FPS = 30

Making classes to represent each entity and define their behaviour in the game

In [4]:
#class for the bird 
class Bird:
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25       #range upto which the player can tilt the bird
    ROT_VEL = 20            #rotation allowed per frame/move
    ANIMATION_TIME = 5      #time for which each animation will hold i.e. the rate at which the bird all flap its wings


    def __init__(self, x, y):
        #starting coordinates for the bird
        self.x = x
        self.y = y
        #starting orientation of the bird
        self.tilt = 0               #the bird looks straight towards the right
        self.tick_count = 0         #tracks the no. of seconds elapsed since the last move. 
                                    #will be used for physics calc for the bird
        self.vel = 0                #bird is stationary
        self.height = self.y
        self.img_count = 0          #image currently being rendered
        self.img = self.IMGS[0]     

    def jump(self):
        self.vel = -10.5            #pygame takes the upper-left corner of the diaplay as (0,0)
                                    #going left increases x, while going down increases y
                                    #therefore, to go up, negative value is required
        
        self.tick_count = 0         #keeps track of when the last jump occured
        self.height = self.y        #height from which the jump was made

    def move(self):
        self.tick_count += 1        #tracks the no. of moves made since the last jump

        #formula defining the arc for the bird when it jumps
        #displacement disp is in pixels
        disp = self.vel*self.tick_count + 1.5*self.tick_count**2

        #setting a limit to the velocity when going downwards and upwards
        if disp > 16:
            disp = 16
        if disp < 0:
            disp -= 2

        self.y = self.y+disp

        if disp < 0 or self.y < self.height + 50:
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION   #while the bird is going up, we don't want it to climb up 90 deg
        else:
            if self.tilt > -90:                 #but while going down, it may look like nose-diving
                self.tilt -= self.ROT_VEL

    def draw(self, win):
        self.img_count += 1

        #decides on which image to show based on the image_count 
        if self.img_count < self.ANIMATION_TIME:
            self.img = self.IMGS[0]       
        elif self.img_count < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0

        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME*2


        #to rotate the image around its center
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center=self.img.get_rect(topleft=(self.x,self.y)).center)
        win.blit(rotated_image, new_rect.topleft)


    #function handling collisions with objects
    def get_mask(self):
        return pygame.mask.from_surface(self.img)

        

In [5]:
#function to render the display window

def draw_window(win, bird):
    win.blit(BG_IMG, (0,0))     #blit() just renders the provided surface onto the display
    bird.draw(win)
    pygame.display.update()

In [6]:
def main():
    bird = Bird(200, 200)
    

    #use clock object to set the tick rate i.e no. of frames per sec
    #prevents the game from using the system's speed and use this measure of time instead
    
    clock = pygame.time.Clock()

    run = True
    #the game loop for the pygame
    while run:
        clock.tick(FPS)      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        
        bird.move()
        draw_window(win, bird)

    pygame.quit()
    quit()


In [7]:
if __name__ == "__main__":
    main()

KeyboardInterrupt: 